In [1]:
import numpy as np
from math import *
from copy import deepcopy
from qiskit import *
import matplotlib.pyplot as plt

Singlet states are states where a linked set of particles have a net angular momentum of zero, or their overall spin quantum number is 0. This result is one spectral line.

Triplet states are states where the overall spin quantum number is 1 and this results in three spectral line.

In [2]:
#Prepared Triplet 0 state:
q = QuantumRegister(2)
c = ClassicalRegister(2)
circT = QuantumCircuit(q,c)
circT.x(0)  #flip second qubit
circT.ry(pi/2, 1) #superposes first qubit
circT.cx(q[1],q[0])

result = execute(circT, backend = BasicAer.get_backend('statevector_simulator')).result()
psi = result.get_statevector(circT)
print('State Vector:' , psi)

State Vector: [0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]


In [3]:
#Preparing Singlet State:
q = QuantumRegister(2)
c = ClassicalRegister(2)

circS = QuantumCircuit(q,c)
circS.x(0)  #flip second qubit
circS.ry(-pi/2, 1) #superposes first qubit
circS.cx(q[1],q[0])

result = execute(circS, backend = BasicAer.get_backend('statevector_simulator')).result()
psi = result.get_statevector(circS)
print('State Vector:' , psi)

State Vector: [ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]


$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$
In order to differentiate the singlet state, $\ket{S} = \frac{1}{\sqrt{2}}(\ket{01} - \ket{10})$, and triplet 0 state, $\ket{T_{1}} = \frac{1}{\sqrt{2}}({\ket{01} + \ket{10}})$, we wrote a function that takes in a circuit and prints whether the circuit was in a singlet or triplet state.

We also realized that this function works for differentiating $\ket{T_{3}} =\frac{1}{\sqrt{2}}({\ket{11} + \ket{00}})$ but does not work for $\ket{T_{2}} = \frac{1}{\sqrt{2}}({\ket{11} - \ket{00}})$ because that would require two measurements.

In [4]:
def singletOrTriplet(circ):
    meas = deepcopy(circ)
    meas.ry(-pi/2, 1)
    meas.ry(-pi/2, 0)
    meas.measure([0,1], [0,1])

    shots = 1
    job = execute(meas, backend = Aer.get_backend('qasm_simulator'), shots = shots)
    counts = job.result().get_counts(meas)
    state = list(counts.keys())[0]
    if state[0] == state[1]:
        print('Triplet State')
    else:
        print('Singlet State')
        
singletOrTriplet(circS)
singletOrTriplet(circT)

Singlet State
Triplet State


***State of 2-qubit system:***
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$

To characterize the state of a 2-qubit system, we would have to find every component of the state vector or the density matrix.

For any 2-qubit system we have the following equation under the standard basis:
$$\ket{\psi} = \alpha_{1}\ket{00} + \alpha_{2}\ket{01} + \alpha_{3}\ket{10} + \alpha_{4}\ket{11}$$
where the state vector is:
$$\ket{\psi} =\begin{bmatrix}   
             \alpha_{1}\\
             \alpha_{2} \\
             \alpha_{3} \\
             \alpha_{4} 
             \end{bmatrix}$$
For a two-qubit system, there will be four complex components to find in the state vector. The way this can be done is by rotating the measurement basis so that you measure the qubits along each basis vector. 

However, it will not always be possible to have complete knowledge of the state, so we can use the density matrix. The density matrix represents the lack of knowledge about the state of a qubit by including all the possible measurement outcomes. This is shown explicitly by:
$$\rho = \Sigma_{i} p_{i}\ket{\psi_{i}}\bra{\psi_{i}}$$

For a single state, the density matrix represents the maximum amount of knowledge one can have of the state.

For example the density matrix of $\ket{1}$ is:
$$\rho = \ket{1}\bra{1} = \begin{bmatrix}   
             0\\
             1
             \end{bmatrix}\begin{bmatrix}   
             0&&1
             \end{bmatrix} = \begin{bmatrix}   
                             0&&0\\
                             0&&1
                             \end{bmatrix}$$

Generally, the density matrix will have a mixture of several projection operators, which represent a mixed state. Projection operators can be thought of as taking an arbitrary mixed state and finding its components of the pure states.

A pure diagonal density matrix with equal matrix elements on the diagonal is referred to as a "completely mixed state". 

The nondiagonal elements of the matrix represent whether the qubits are in a superposition.

The density matrix must have the following properties:

   - $Tr(\rho) = 1$ (Values along the diagonal are real)

   - The eigenvalues of the density matrix are positive and lie between 0 and 1. (When $\lambda = 1$, the vector is along one of the basis vectors with 100% probability.) 
   
   - Eigenvalues are the expectation values of the different states.

   - Since the matrix is Hermtian the triangular components are complex conjugates of each other.
   
   - For a pure state: 
   $$\rho^{2} = \rho$$
   $$Tr(\rho^{2}) = 1$$
   
   - For a mixed or entangled state:
   $$\rho^{2} \neq \rho$$
   $$Tr(\rho^{2}) < 1$$
   
For a two-qubit system we have the following density matrix:
$$\begin{bmatrix}   
             a_{0}&&(b_{0}+b_{1}i)&&(c_{0}+c_{1}i)&&(d_{0}+d_{1}i)\\
             (b_{0}-b_{1}i)&&a_{1}&&(e_{0}+e_{1}i)&&(f_{0}+f_{1}i) \\
             (c_{0}-c_{1}i)&&(e_{0}-e_{1}i)&&a_{2}&&(g_{0}+g_{1}i) \\
             (d_{0}-d_{1}i)&&(f_{0}-f_{1}i)&&(g_{0}-g_{1}i)&& a_{3} 
             \end{bmatrix}$$
Considering these properties of the density matrix, we have 15 unknown parameters. Quantum State Tomography can be used to find these unknown parameters and construct the density matrix.

The number of parameters goes as $4^{n} - 1$ where n is the number of qubits.